In [ ]:
from google.colab import drive
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics
from keras.preprocessing.text import Tokenizer                    
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers
from sklearn.metrics import f1_score

read dataset

In [ ]:
drive.mount('/content/drive')
os.chdir('drive/MyDrive/CS4248 Project')

Mounted at /content/drive


In [ ]:
train = pd.read_csv('raw_data/fulltrain.csv')
train.columns =['Label', 'Text']
test = pd.read_csv('raw_data/balancedtest.csv')
test.columns = ['Label','Text']

In [ ]:
train['Label'].value_counts()


3    17870
1    14046
4     9995
2     6942
Name: Label, dtype: int64

In [ ]:
class2 = train.loc[train['Label'] == 2]
class4 = train.loc[train['Label'] == 4]
class1 = train.loc[train['Label'] == 1]
class3 = train.loc[train['Label'] == 3]

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from nltk.tokenize import sent_tokenize
import random
shuffle_sent2 = []
for i in range(class2.shape[0]):
  a = sent_tokenize(class2['Text'].iloc[i])
  random.shuffle(a)
  a = ' '.join(a)
  shuffle_sent2.append(a)
class2_sent = class2.drop("Text",axis =1 )
class2_sent['Text'] = shuffle_sent2

In [ ]:
shuffle_sent2 = []
for i in range(class2.shape[0]):
  a = sent_tokenize(class2['Text'].iloc[i])
  random.shuffle(a)
  a = ' '.join(a)
  shuffle_sent2.append(a)
class2_sent = class2.drop("Text",axis =1 )
class2_sent['Text'] = shuffle_sent2

In [ ]:
shuffle_sent4 = []
for i in range(class4.shape[0]):
  a = sent_tokenize(class4['Text'].iloc[i])
  random.shuffle(a)
  a = ' '.join(a)
  shuffle_sent4.append(a)
class4_sent = class4.drop("Text",axis =1 )
class4_sent['Text'] = shuffle_sent4

In [ ]:
shuffle_sent3 = []
for i in range(class3.shape[0]):
  a = sent_tokenize(class3['Text'].iloc[i])
  random.shuffle(a)
  a = ' '.join(a)
  shuffle_sent3.append(a)
class3_sent = class3.drop("Text",axis =1 )
class3_sent['Text'] = shuffle_sent3

In [ ]:
shuffle_sent1 = []
for i in range(class1.shape[0]):
  a = sent_tokenize(class1['Text'].iloc[i])
  random.shuffle(a)
  a = ' '.join(a)
  shuffle_sent1.append(a)
class1_sent = class1.drop("Text",axis =1 )
class1_sent['Text'] = shuffle_sent1

In [ ]:
from nltk.tokenize import word_tokenize
shuffle_word2 = []
for i in range(class2.shape[0]):
  a = word_tokenize(class2['Text'].iloc[i])
  random.shuffle(a)
  a = ' '.join(a)
  shuffle_word2.append(a)
class2_word = class2.drop("Text",axis =1 )
class2_word['Text'] = shuffle_word2

In [ ]:
train = train.append(class4_sent.sample(n=8500))
train = train.append(class2_sent.sample(n=6000))
train = train.append(class2_word.sample(n=5500))
train = train.append(class3_sent.sample(n=500))
train = train.append(class1_sent.sample(n=4000))

In [ ]:
train['Label'].value_counts()


4    18495
2    18442
3    18370
1    18046
Name: Label, dtype: int64

Set weight for each type to make them balanced

The original train text.

In [ ]:
train_x = train['Text'].values
test_x = test['Text'].values


### create dataset.

In [ ]:
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(train_x)

In [ ]:
X_train = tokenizer.texts_to_sequences(train_x)
X_test = tokenizer.texts_to_sequences(test_x)

Padding data

In [ ]:
vocab_size = len(tokenizer.word_index) + 1                          

maxlen = 100

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

word2vec embedding.

In [ ]:
def create_embedding_matrix(filepath, word_index, embedding_dim):
    vocab_size = len(word_index) + 1  
    # Adding again 1 because of reserved 0 index
    embedding_matrix = np.zeros((vocab_size, embedding_dim))

    with open(filepath) as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word] 
                embedding_matrix[idx] = np.array(vector, dtype=np.float32)[:embedding_dim]

    return embedding_matrix

Golve word2vec package

In [ ]:
embedding_dim = 200
embedding_matrix = create_embedding_matrix('glove.6B.200d.txt' ,
                                            tokenizer.word_index,  
                                            embedding_dim)

Create label for model, train_y.shape[48853,4]

In [ ]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')
enc_df = enc.fit_transform(train[['Label']]).toarray()
enc_df2 = enc.fit_transform(test[['Label']]).toarray()
train_y = pd.DataFrame(enc_df)
test_y = pd.DataFrame(enc_df2)

### CNN model. 

In [ ]:
from keras.models import Sequential
from keras import layers
from sklearn.metrics import fbeta_score
embedding_dim = 200

model = Sequential()
model.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))
model.add(layers.GaussianNoise(stddev=0.2))
model.add(layers.Conv1D(128, 5, activation='relu',kernel_regularizer=tf.keras.regularizers.l2( l2 = 0.1)))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dropout(0.3))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(4, activation='softmax'))
model.compile(optimizer='Adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model_checkpoint = tf.keras.callbacks.ModelCheckpoint("CNN_model.h5", save_best_only=True, monitor='val_accuracy', verbose=1)
cnn = model.fit(X_train, train_y,
                    epochs=15,
                    shuffle = True,
                    validation_data=(X_test, test_y),
                    batch_size=32,
                    callbacks=[model_checkpoint])

Epoch 1/100
1605/1605 [==============================] - 104s 57ms/step - loss: 1.2646 - precision: 0.8330 - recall: 0.5223 - accuracy: 0.6621 - val_loss: 1.0982 - val_precision: 0.8196 - val_recall: 0.6984 - val_accuracy: 0.7652
Epoch 2/100
1605/1605 [==============================] - 92s 57ms/step - loss: 0.8074 - precision: 0.9057 - recall: 0.8710 - accuracy: 0.8882 - val_loss: 0.9729 - val_precision: 0.8736 - val_recall: 0.8065 - val_accuracy: 0.8453
Epoch 3/100
1605/1605 [==============================] - 91s 56ms/step - loss: 0.7450 - precision: 0.9267 - recall: 0.9042 - accuracy: 0.9147 - val_loss: 0.9036 - val_precision: 0.8644 - val_recall: 0.8056 - val_accuracy: 0.8351
Epoch 4/100
1605/1605 [==============================] - 90s 56ms/step - loss: 0.6940 - precision: 0.9338 - recall: 0.9157 - accuracy: 0.9247 - val_loss: 1.0503 - val_precision: 0.7868 - val_recall: 0.7459 - val_accuracy: 0.7671
Epoch 5/100
1605/1605 [==============================] - 90s 56ms/step - loss: 0.64